In [1]:
import string
import time
from nltk.util import ngrams
from nltk.corpus import stopwords
import nltk
from collections import OrderedDict
from collections import Counter
from collections import defaultdict


## Preprocessing the tokens

In [2]:
def preprocess(tokenList):
    i=0
    for word1 in tokenList:
    #conversion into lowercase
        word1=word1.lower()
    #Takes Care of Multiple Punctuation Marks
        word1=word1.replace('.','').replace(',','').replace(':','').replace(';','').replace('!','').replace('?','').replace('(','').replace(')','').replace('-','').replace('_','').replace('\\',' ').replace('\"',' ').replace('\'',' ')      
    
        tokenList[i]=word1        
        i=i+1

## Creating unigram dictionary

In [3]:
def unigramize(unigrams):
    
    od=Counter()
    
    for item in unigrams:
        od[item]+=1
        
    return od

## Creating bigram dictionary

In [4]:
def bigramize(bigrams):
    od2=Counter()


    for item in bigrams:
        od2[item]+=1
        
    return od2

## Creating trigram dictionary

In [5]:
def trigramize(trigrams):
    od3=Counter()


    for item in trigrams:
        od3[item]+=1
        
    return od3

## Creating quadgram dictionary

In [6]:
def quadgramize(quadgrams):
    od4=Counter()

    for item in quadgrams:
        od4[item]+=1
        
    return od4

In [7]:
def uni_prob(token,unigrams):
    
    uni_freq=Counter(unigrams)
    n=len(token)
    
    for item in uni_freq:
        uni_freq[item]=uni_freq[item]/n
        
    return uni_freq
    
    

In [8]:
def bi_prob(unigrams,bigrams):
    
    uni_freq=Counter(unigrams)
    bi_freq=Counter(bigrams)
    
    for item in bi_freq:
        uni=item[0]
        bi_freq[item]=bi_freq[item]/uni_freq[uni]
        
    return bi_freq

In [9]:
def tri_prob(bigrams,trigrams):
    
    bi_freq=Counter(bigrams)
    tri_freq=Counter(trigrams)
    
    for item in tri_freq:
        bi=item[0:2]
        tri_freq[item]=bi_freq[item]/bi_freq[bi]
        
    return tri_freq

## Creating quadgram probability table

In [10]:
def quad_prob(trigrams,quadgrams):
    
    tri_freq=Counter(trigrams)
    quad_freq=Counter(quadgrams)
    #print (tri_freq.items())
    
    for item in quad_freq:
        tri=item[0:3]
        quad_freq[item]=(quad_freq[item]/tri_freq[tri])
        
    return quad_freq

## Predicting the word

In [11]:
def pred_table(quad_prob_table):
    quad_pred_table=defaultdict(dict)
    
    for quad in quad_prob_table:
        prob=quad_prob_table[quad]
        tri=quad[0:3]
        token=quad[3]
        quad_pred_table[tri][token]=prob
        
    for tri in quad_pred_table:
        quad_pred_table[tri]=sorted(quad_pred_table[tri].items(), key=lambda x: x[1], reverse=True)
        
        return quad_pred_table
        
    


## Interpolation using given weights

In [12]:
def interpolation_table(od,od2,od3,od4):
    
    pole=defaultdict(dict)
    
    i=0
    s=0.0
    lambda1=0.25
    lambda2=0.25
    lambda3=0.25
    lambda4=0.25
    
    for item,value in od4.items():
        #if i>100:
         #   break    
        p=lambda1*float((value)/od3[item[0:3]]) + lambda2*float((od3[item[1:4]])/(od2[item[1:3]])) + lambda3*float((od2[item[2:4]])/(od[item[2]]+1)) +lambda4*(float((od[item[3]]+1)))
        pole[item[0:3]][item[3]]=p    
        
    for tri in pole:
        pole[tri]=sorted(pole[tri].items(), key=lambda x: x[1], reverse=True)
        
    return pole

## Interpolation using customised weights

In [13]:
def cus_interpolation_table(od,od2,od3,od4,l):

    pole=defaultdict(dict)
    pmax=0
    i=0
                         
    for item,value in od4.items(): 
        i=i+1
        if (i>100):
            break
        l=weights(l)
        p=l[0]*float((value)/od3[item[0:3]]) + l[1]*float((od3[item[1:4]])/(od2[item[1:3]])) + l[2]*float((od2[item[2:4]])/(od[item[2]]+1)) +l[3]*(float((od[item[3]]+1)))
        if(p>pmax):
            pole[item[0:3]][item[3]]=p
            pmax=p
                                
    
    for tri in pole:
        pole[tri]=sorted(pole[tri].items(), key=lambda x: x[1], reverse=True)
        
    return pole

In [14]:
def weights(l):
    
    for i in xfrange (0.01,1,0.01):
            for j in xfrange (0.01,1,0.01):
                for k in xfrange (0.01,1,0.01):
                    for m in xfrange (0.01,1,0.01):
                        if (i+j+k+m==1):
                            l.append(i)
                            l.append(j)
                            l.append(k)
                            l.append(m)
                            break
                            
    return l
                            

In [15]:
def xfrange(start, stop, step):
    i = 0
    while start + i * step < stop:
        yield start + i * step
        i += 1

## Calculating score of the model

In [16]:
def scoreCalc(quad,tri,tokenList2):
    score=0
    scorepred=OrderedDict()
    scorepred=OrderedDict.fromkeys(tokenList2,0)

    for item in quad:
        if item[0:3] in tri:
            scorepred[item[3]]+=1
        v=list(scorepred.values())
        k=list(scorepred.keys())
        if (k[v.index(max(v))]==item[3]):
            score+=1

    return score

## Add k smoothing

In [17]:
def trismoothk(tokens,tri,trigramSet2,smooth_tri,k):
    
    #Add k Smoothing for trigram model
    
    i=len(trigramSet2)
    
    for item in trigramSet2:
        smooth_tri[item]=tri.count(item) + k/float((tokens.count(item[0:2])+ i))
        

In [18]:
def quadsmoothk(tokens,quad,quadgramSet2,smooth_quad,k):
    
    #Add k Smoothing for quadgram model
    
    i=len(quadgramSet2)
    
    for item in quadgramSet2:
        smooth_quad[item]=quad.count(item) + k/float((tokens.count(item[0:3])+ i))
          
    


## Good Turing Smoothing

In [19]:
def goodturing(quadgramSet2,wcount):
    
    n1=0
    n2=0
    n=len(quadgramSet2)
    
    wprob=defaultdict(dict)
    
    
    for item in wcount:
        if(wcount[item]==1):
            n1=n1+1
        if(wcount[item]==2):
            n2=n2+1
    
    for item in quadgramSet2:
        if (wcount[item[3]]==0):
            wcount[item[3]]=float(n1/n)
        if (wcount[item[3]]==1):
            wcount[item[3]]=(2*float(n2/n1))/n
        else:
            wcount[item[3]]=wcount[item[3]]/n
        tri=item[0:3]
        prob=wcount[item[3]]
        wprob[tri][item[3]]=prob
        
    for tri in wprob:
        wprob[tri]=sorted(wprob[tri].items(), key=lambda x: x[1], reverse=True)
        
            
    return wprob
            
            
            
        

## Kneser Ney Smoothing

In [20]:
def kney(od3,od4):
    
    kneser=defaultdict(dict)
    d=0.25
    lambda1=0.75
    n1=0
    n2=0
    
    for item,val1 in od4.items():
        disc=float((od4[item]-d)/od3[item[0:3]])
        #for tri,val2 in od3.items():
         #   if (tri[2]==item[3]):
        n1=od3[item[0:3]]
        n2=od4[item]
        cont=float(lambda1*n1/n2)
        prob=disc+cont
        kneser[item[0:3]][item[3]]=prob
        
    for tri in kneser:
        kneser[tri]=sorted(kneser[tri].items(), key=lambda x: x[1], reverse=True)
        
    return kneser
        
    

## Perplexity

In [21]:
def quadperp(tokenList2,smooth_quad,smooth_tri,quadgramSet2):
    
    #computing quadgram perplexity
    n= len (tokenList2)
    perplexity4=1.0
    
    for item in quadgramSet2:
        perplexity4=perplexity4*(((1/float(smooth_quad[item]))*smooth_tri[item[0:3]])**(1./n))
        
    print ("Quadgram Perplexity = %f" %(perplexity4))


## Partitioning testing corpus to create the held-out set

In [22]:
#loads the testing corpus and partitions it into equal halves to create testing corpus and held-out corpus
def partitionFile(file_path,word_count):
    token = []
    word_count = int(word_count*0.5)
      
    pos = 0
    word_len = 0

    #open the corpus file and read it line by line
    file = open(file_path,'r')
    test_file = open('testing_corpus.txt','w')
    held_file = open('heldset_corpus.txt','w') 

    line = file.readline()

    while line:
        #split the line into tokens
        token = line.split()

        #write the line to the training file
        test_file.write(line)

        word_len = word_len + len(token)  
            
        #quit training when 90% of the corpus has been read
        if word_len >= word_count:
                pos = file.tell()
                break;
            
        line = file.readline()

    #Prepare the testing data
    if word_count <= word_len:
        file.seek(pos)
        test_data = file.read();
        held_file.write(test_data)
            
    file.close()
    test_file.close()
    held_file.close()

In [42]:
def backoff(tokens,test_quadgrams,train_quadgrams,train_trigrams,train_bigrams,train_unigrams):
    
    for item in test_quadgrams:
        print (item)
        if(train_quadgrams[item]>0):
            print ("probability with quadgram %f"
                  %((train_quadgrams[item])/float(train_trigrams[item[0:3]])))
        
        elif(train_trigrams[item[0:3]]>0):
            print ("probability with trigram %f"
                  %((train_trigrams[item[1:4]])/float(train_bigrams[item[1:3]])))
            
        elif(train_bigrams[item[1:3]]>0):
            print ("probability with bigram %f"
                  %((train_bigrams[item[2:4]])/float(train_unigrams[item[2]]+1)))
        elif(train_bigrams[item[2:4]]<=0):
            print ("Probability with unigram %f" %((train_unigrams[item[3]])/float(len(tokens))))
                    
                
        

## Main function calling all modules

In [52]:
def main():
    f=open('Data/LanguageModels/training_corpus.txt','r',encoding='latin1')
    content=f.read()
    token=content.split()

    #splitting into tokens
    tokenList=list(token)
    tokenSet=set(tokenList)

    #storing stop words separately
    stop = set(stopwords.words('english'))
    #print (len(stop))
    #tokenSet=tokenSet-stop


    #listing the tokens into n-grams
    unigrams=list(ngrams(token,1))
    bigrams=list(ngrams(token,2))
    trigrams=list(ngrams(token,3))
    quadgrams=list(ngrams(token,4))
    
    #Preprocessing the training set
    preprocess(tokenList)
    
    od=Counter()
    od=unigramize(unigrams)
    
    od2=Counter()
    od2=bigramize(bigrams)
    
    od3=Counter()
    od3=trigramize(trigrams)
    
    
    od4=Counter()
    od4=quadgramize(quadgrams)
    
    
    
    #Taking our input string
    sent=input("Enter your test string: ")
    list2=sent.split()
    #test=' '.join(list2[(len(list2)-3):len(list2)])
    sent_tri=list(ngrams(list2,3))
    x=len(sent_tri)
    test_tri=sent_tri[x-1]
    

    #Normal prediction of the most probable word
    time1=time.time()
    quad_prob_table=Counter(od4)
    quad_prob_table=quad_prob(od3,od4)
    quad_pred_table=defaultdict(dict)
    quad_pred_table=pred_table(quad_prob_table)
    word=quad_pred_table[test_tri]
    word=sorted(word.items(), key=lambda x: x[1], reverse=True)
    print ("The next word could be: ")    
    print (word[0][0])
    print ("Time taken for prediction: ", time.time()-time1)
    
    #Prediction of the word after interpolation using given weights
    time1=time.time()
    quad_pole_table=defaultdict(dict)
    quad_pole_table=interpolation_table(od,od2,od3,od4)
    word=quad_pole_table[test_tri]
    word=sorted(word, key=lambda x: x[1], reverse=True)
    print ("After interpolation, the most probable word could be: ")
    print (word[0][0])
    print ("Time taken for prediction: ", time.time()-time1)
    
    
    with open('Data/LanguageModels/testing_corpus.txt','r',encoding='latin1') as f:
            contents=f.read()
            tokens=contents.split()
            tokenList2=list(tokens)
            
    #creating the held-out set for interpolation
    partitionFile('Data/LanguageModels/testing_corpus.txt',len(tokens))
    
    
    with open('Data/LanguageModels/testing_corpus.txt','r',encoding='latin1') as fheld:
            hcontents=fheld.read()
            htokens=contents.split()
            htokenList=list(htokens)
     
    #Preprocessing the test set and held-out set
    preprocess(tokens)
    preprocess(htokens)
    
    bi=list(ngrams(tokenList2,2))
    tri=list(ngrams(tokenList2,3))
    quad=list(ngrams(tokenList2,4))
    
    hbi=list(ngrams(htokenList,2))
    htri=list(ngrams(htokenList,3))
    hquad=list(ngrams(htokenList,4))
    
    bigramSet2=set(bi)
    trigramSet2=set(tri)
    quadgramSet2=set(quad)
    hod=Counter()
    hod=unigramize(htokens)
    hod2=Counter()
    hod2=bigramize(hbi)
    hod3=Counter()
    hod3=trigramize(htri)
    hod4=Counter()
    hod4=quadgramize(hquad)
   
    
    #Prediction of the word after interpolation using customised weights
    time1=time.time()
    quad_cpole_table=defaultdict(dict)
    l=[]
    #quad_cpole_table=cus_interpolation_table(hod,hod2,hod3,hod4,l)
    #word=quad_cpole_table[test_tri]
    #word=sorted(word, key=lambda x: x[1], reverse=True)
    #print ("After interpolation, the most probable word could be: ")
    #print (word)
    #print (l)
    #print ("Time taken for prediction: ", time.time()-time1)
    
    
    #alculating Score of the language model
    time1=time.time()
    #print ("Score of the language model is: ")
    #print (scoreCalc(quad,tri,tokenList2))
    #print ("Time taken for calculating Score: ", time.time()-time1)
    
    
    #Add K Smoothing
    smooth_tri=OrderedDict()
    smooth_tri=OrderedDict.fromkeys(trigramSet2,0)
    smooth_quad=OrderedDict()
    smooth_quad=OrderedDict.fromkeys(quadgramSet2,0)
    #k=int(input("We will perform Add k smoothing now. Enter value of k: "))
    time1=time.time()
    #trismoothk(tokens,tri,trigramSet2,smooth_tri,k)
    #quadsmoothk(tokens,quad,quadgramSet2,smooth_quad,k)
    #print ("Time taken for Add k Smoothing: ", time.time()-time1)
    
    #Good Turing Smoothing
    od=unigramize(tokens) 
    wprob=defaultdict(dict)
    time1=time.time()
    #wprob=goodturing(quadgramSet2,od)
    word=wprob[test_tri]
    word=sorted(word, key=lambda x: x[1], reverse=True)
    #print ("After Good Turing Smoothing, the most probable word could be: ")
    #print (word[0][0])
    #print ("Time taken for Good Turing Smoothing: ", time.time()-time1)
    
    #Kneser Ney Smoothing
    wprob=defaultdict(dict)
    time1=time.time()
    #wprob=kney(od3,od4)
    word=wprob[test_tri]
    word=sorted(word, key=lambda x: x[1], reverse=True)
    #print ("After Kneser Ney Smoothing, the most probable word could be: ")
    #print (word[0][0])
    #print ("Time taken for Kneser Ney Smoothing: ", time.time()-time1)
    
    
    #Perplexity
    time1=time.time()
    #quadperp(tokenList2,smooth_quad,smooth_tri,quadgramSet2)
    #print ("Time taken to compute perplexity: ", time.time()-time1)
    
    #Backoff
    time1=time.time()
    #backoff(tokens,quad,od4,od3,od2,od)
    #print ("Time taken to compute Backoff: ", time.time()-time1)
    
    
    
    

In [53]:
if __name__ == "__main__":
    main()

Enter your test string: I could not
The next word could be: 
have
Time taken for prediction:  0.3926658630371094
After interpolation, the most probable word could be: 
be
Time taken for prediction:  1.4052565097808838
('If', 'you', 'need', 'farther')
probability with bigram 0.000000
('you', 'need', 'farther', 'explanation,')
Probability with unigram 0.000000
('need', 'farther', 'explanation,', 'I')
Probability with unigram 0.000000
('farther', 'explanation,', 'I', 'have')
('explanation,', 'I', 'have', 'the')
probability with bigram 0.210884
('I', 'have', 'the', 'honour,')
probability with trigram 0.000000
('have', 'the', 'honour,', 'my')
probability with bigram 0.000000
('the', 'honour,', 'my', 'dear')
('honour,', 'my', 'dear', 'madam,')
probability with bigram 0.000000
('my', 'dear', 'madam,', 'of')
Probability with unigram 0.027059
('dear', 'madam,', 'of', 'being')
('madam,', 'of', 'being', 'your')
probability with bigram 0.000000
('of', 'being', 'your', "husband's")
Probability with